<a href="https://colab.research.google.com/github/tchintchie/email_scraper/blob/master/Email_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Step 1: Import Libraries
---
credit goes to https://medium.com/swlh/how-to-scrape-email-addresses-from-a-website-and-export-to-a-csv-file-c5d1becbd1a0


In [92]:
import re
import requests
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup
import pandas as pd

## Step 2: Initialize variables

Then, we initialize a `deque` for saving unscraped URLs, a `set` for scraped URLs, and a `set` for saving emails scraped successfully from the website.

In [93]:
original_url = "http://www.auderer.at/"
#original_url = input("Enter URL to scrape: ")

unscraped_url = deque([original_url])
scraped_url = set()
scraped_mails = set()

## Step 3: Start scraping
First, move a url from unscraped to scraped.

In [ ]:
while len(unscraped_url):
  url = unscraped_url.popleft()
  scraped_url.add(url)
  parts = urlsplit(url)
  path = url
  base_url = "{0.scheme}://{0.netloc}".format(parts)
  if "/" in parts.path:
    path = url[:url.rfind('/')+1]
  else:
    path = url
  print("Crawling URL %s ..." %url)
  try:
    respones = requests.get(url)
  except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
    continue
  new_email = set(re.findall(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", respones.text, re.I))
  scraped_mails.update(new_email)
  soup = BeautifulSoup(respones.text, "lxml")
  for a in soup.find_all("a"):
    if "href" in a.attrs:
      link = a.attrs["href"]
    else:
      link = ""
    if link.startswith("/"):
      link = base_url + link
    elif not link.startswith("http"):
      link = path + link

    if not link.endswith(".gz"):
      if not link in unscraped_url and not link in scraped_url:
        unscraped_url.append(link)


Crawling URL http://www.auderer.at/ ...
Crawling URL http://www.auderer.at/de/ ...
Crawling URL http://www.auderer.at/en/ ...
Crawling URL http://maps.imst.at/ ...
Crawling URL http://www.auderer.at/# ...
Crawling URL http://www.auderer.at/index.php ...
Crawling URL http://www.auderer.at/wordpress/de/ ...
Crawling URL http://www.auderer.at/de/category/hotel/ ...
Crawling URL http://www.auderer.at/de/top-angebote/ ...
Crawling URL http://www.auderer.at/de/category/hotel/ambiente/ ...
Crawling URL http://www.auderer.at/de/category/hotel/firmen-tagungen/ ...
Crawling URL http://www.auderer.at/de/category/hotel/lage-anreise/ ...
Crawling URL http://www.auderer.at/de/category/hotel/busreisen/ ...
Crawling URL http://www.auderer.at/de/category/impressionen/ ...
Crawling URL http://www.auderer.at/de/zimmer/ ...
Crawling URL http://www.auderer.at/de/category/zimmer/einzelzimmer/ ...
Crawling URL http://www.auderer.at/de/category/zimmer/doppelzimmer/ ...
Crawling URL http://www.auderer.at/de/ca

In [ ]:
scraped_mails